In [ ]:
import math as ms
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
plt.rc('font',family='Times New Roman') 
from scipy import stats as st
import torch
import csv
from sklearn.manifold import TSNE
import torch.nn as nn
import torch.nn.functional as F
from numpy import random
from d2l import torch as d2l
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import os
from matplotlib.animation import FuncAnimation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sim =50
# Sim is W in the paper
def h(t,ts,dt):
    rd = 5*1e-6
    Da = 5*1e-9
    Dr1 = 1*1e-13
#     dt = 15*1e-6
    Vd = 4/3*np.pi*pow(rd,3)
    D1 = Da + Dr1
    return Vd/pow(4*np.pi*D1*ts,3/2)*np.exp(-pow(dt,2)/(4*D1*ts))

def noise(y,SNR):
    for i in range(len(y)):
        A = torch.sum(y[i,:-1] ** 2)/len(y[i,:-1])
        sigma = np.sqrt(A/(np.power(10,(SNR/10))))
        y[i,:-1]+=np.random.normal(0,sigma,1)
    return y
def Receive(j,X,dt,su,k):
    mjs = su
    Ts =35*1e-3#[100,500]
    ts = Ts /sim #ms
    st= 0

    if X[j] == 0:
        if j > 3:
            for i in range(j-3,j):
                st = st+sum(random.poisson(mjs*X[i]*h(i*Ts,(j-i)*Ts+k*ts,dt),1))
            return st
        else:
            for i in range(0,j):
                st = st+sum(random.poisson(mjs*X[i]*h(i*Ts,(j-i)*Ts+k*ts,dt),1))
            return st
        
    if X[j] == 1:
        if j > 3:
            for i in range(j-4,j):
                st = st+sum(random.poisson(mjs*X[i]*h(i*Ts,(j-i)*Ts+k*ts,dt),1))
            ot =st+sum(random.poisson(mjs*h(j*Ts,ts,dt),1))
            return ot
        else:
            for i in range(0,j):
                st = st+sum(random.poisson(mjs*X[i]*h(i*Ts,(j-i)*Ts+k*ts,dt),1))
            ot =st+sum(random.poisson(mjs*h(j*Ts,ts,dt),1))
            return ot
    
def Receivemu_Y(j,n,distance,inputX,su,k):
    at=0
    k_i = 1
    for i in range(1,n):
        at = at+k_i*float(Receive(j,inputX,sum(distance[i-1:]),su,k))
        
    return at
def Receivemu_Y_bd(n,d,su,SNR):
#     n is the number of nodes in the communication model, where it is 2
#     d is the distance between two adjacent nodes
#     su is the number of molecules released by the sender TN when sending signal 1
#     train_data = Receivemu_Y_bd(k,[j*1e-6]*(k-1),10000,n)
    epoch = 101
#     d = [10*1e-6,10*1e-6]
#     L = len(d)
    L=1
    x = random.randint(0,2,int(epoch-1))
    sjs = 4
    y = torch.zeros(epoch*L-1,sjs*sim).to(device)

    x=np.insert(x,0,[0,0,0])
    begin = time.time()
    for k in range(L):
        for j in range(3,epoch+1):
            for i in range(0,sim):
                y[j+k*epoch-3,i] = Receivemu_Y(j-3,n,d,x,su,i)
            for i in range(sim,2*sim):
                y[j+k*epoch-3,i] = Receivemu_Y(j-2,n,d,x,su,i-sim)
            for i in range(2*sim,3*sim):
                y[j+k*epoch-3,i] = Receivemu_Y(j-1,n,d,x,su,i-sim*2)
            for i in range(3*sim,4*sim):
                y[j+k*epoch-3,i] = Receivemu_Y(j,n,d,x,su,i-sim*3)
#             y[j+k*epoch-4,-1] = d[k]*10
    end = time.time()
    y = y.cpu()
#     y = y[~(y==0).all(1)]
    x = np.tile(x[3:],(1,L))
    train_data = torch.cat((y,torch.tensor(x).reshape((epoch-1)*L,1)),1)
    train_data = noise(train_data,SNR)
    train_data=torch.clamp(train_data,min=0.0)
    print("{:5.2f}s".format(begin - end))
    return train_data

